**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
from random import sample,choice

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function 'act' determines the next action that the algorithm will take given the current state. The algorithm chooses between exploration or exploitation based on the value of epsilon. Epsilon controls how much the algorithm should rely on past information. If the random integer generated is less than epsilon, exploration is done and a random action from 1 to 4 is chosen. However if not, then the action taken is based on the defined policy(exploitation). The value of epsilon is critical as it defines the trade off between exploration and exploitation. Setting a low value would generate low probability of exploration and higher exploitation. This would be ideal in a case where the optimal policy has been defined. However, if we are still tuning the policy it might be better to balance both exploration and exploitation in order to implement a better algorithm. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        """ why +4 for grid size """
        grid_size = grid_size+4 
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        
        """"R- Possibly setting the edges of the board with a penalty of -1 to teach the algorithm that it is the corner of the board  """
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        """R-Setting position of the cat"""
        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        #print("State shape is ",state.shape)
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Board is used to set the reward template of the board. It has values corresponding to the reward (bonus if cheese is present and negative score for the poisonous cells). Whenever the rat moves to a cell on the board with cheese/poison present, the cell value on the board is then set to 0. This is how the game progresses. 

Position array is used to trace the position of the rat. The cell where the rat is present is given a value of 1. Tthe positions beyond the board which the rat can see but can't go are given a -1 and where it can move is given a 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        """Pasting Agent constructor for quick reference 
        def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action """
        pass

    def learned_act(self, s):
        #Random Agent would move randomly irrespective of state. only exploration
        a = np.random.randint(0, self.n_action, size=1)[0]   
        return a
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
         ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose-reward
                 
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [47]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.5/10.0. Average score (-4.5)
Win/lose count 10.0/21.0. Average score (-7.75)
Win/lose count 7.5/11.0. Average score (-6.333333333333333)
Win/lose count 10.0/15.0. Average score (-6.0)
Win/lose count 8.0/11.0. Average score (-5.4)
Win/lose count 10.0/12.0. Average score (-4.833333333333333)
Win/lose count 12.0/14.0. Average score (-4.428571428571429)
Win/lose count 14.5/13.0. Average score (-3.6875)
Win/lose count 12.5/14.0. Average score (-3.4444444444444446)
Win/lose count 12.5/16.0. Average score (-3.45)
Final score: -3.45


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





1. 
$Q^\pi(s,a) = E_\pi \Big[ \overset{\infty}{\underset{k=0}{\sum}} \gamma^k r_{t+k} \Big| s_t = s, a_t = a \Big]$

$= E\Big[r(s,a) + \overset{\infty}{\underset{k=1}{\sum}} \gamma^k r_{t+k} \Big| s_t = s, a_t = a\Big]$

$ =  r(s,a) + \gamma E\left[E\left[\overset{\infty}{\underset{k=0}{\sum}}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a'\right]|s_t = s, a_t = a\right]$

$ = r(s,a) + \gamma \underset{(s',a')}{\sum}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ))*E[\overset{\infty}{\underset{k=0}{\sum}}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']$

$ =  \underset{(s',a')}{\sum} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \underset{(s',a')}{\sum}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ))E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']$

$= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$

$~$
$~$

2. 
$Q^*(s,a) = \underset{(a',\pi')}{\max}\left[E_{(s',a')\sim p(.|s,a)}\left(r(s,a)+\gamma Q^{(a',\pi')}(s',a')\right)\right] $

$ = \underset{(a',\pi')}{\max}\left[r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right]$

$= r(s,a) + \gamma * \underset{(a',\pi')}{\max}\left[ \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right] $

$~$ $~$

3. Rewriting $Q^{*}(s,a)$ as

$Q^{*}(s,a) = r(s,a) + \gamma  \overset{\infty}{\underset{k=0}{\sum}}(p(s_{t+1}=s' |s_t = s, a_t = a )\underset{a'}{\max}\underset{\pi'}{\max}Q^{\pi'}(s',a') $ $= r(s,a) + \gamma \overset{\infty}{\underset{k=0}{\sum}}(p(s_{t+1}=s' |s_t = s, a_t = a )\underset{a'}{\max} Q^{*}(s',a') $ $=E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\underset{a'}{\max}Q^{*}(s',a')\right]$
$~$ $~$

Objective function
$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}*\Vert r+\gamma\max_{a'}\left(Q(s',a',\theta)-Q(s,a,\theta)\right)\Vert^{2}$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        #Function appends each new move to the list 'memory' based on the available memory
        #If memory is overshot, then delete the oldest move and append new one 
        self.memory.append(m)  
        if len(self.memory)>self.max_memory:
            self.memory.pop(0)
        else:
            pass

    def random_access(self):
        rand_loc = np.random.randint(len(self.memory), size=1)[0]
        return self.memory[rand_loc]
        pass

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = np.reshape(s,[1,5,5,self.n_state])
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))        
              
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_=self.memory.random_access()
            s_ = np.reshape(s_,[1,5,5,self.n_state])
            n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
            target_q[i] =self.model.predict(s_)[0]
            input_states[i] = s_
            if game_over_:
                ######## FILL IN
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                target_q[i,a_] = (r_ + (self.discount * np.amax(self.model.predict(n_s_)[0])))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
       
        # NN Model
        model=Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        #model.add(Dense(256,activation ='relu'))
        model.add(Dense(128,activation ='relu'))
        model.add(Dense(4))
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model

In [86]:
epochs_train=20
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0038 | Win/lose count 7.0/4.0 (3.0)
Epoch 001/020 | Loss 0.0073 | Win/lose count 3.5/2.0 (1.5)
Epoch 002/020 | Loss 0.0030 | Win/lose count 2.0/5.0 (-3.0)
Epoch 003/020 | Loss 0.0041 | Win/lose count 3.0/8.0 (-5.0)
Epoch 004/020 | Loss 0.0046 | Win/lose count 4.5/4.0 (0.5)
Epoch 005/020 | Loss 0.0268 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/020 | Loss 0.0022 | Win/lose count 6.5/5.0 (1.5)
Epoch 007/020 | Loss 0.0049 | Win/lose count 4.5/5.0 (-0.5)
Epoch 008/020 | Loss 0.0044 | Win/lose count 9.0/1.0 (8.0)
Epoch 009/020 | Loss 0.0101 | Win/lose count 9.5/1.0 (8.5)
Epoch 010/020 | Loss 0.0060 | Win/lose count 11.0/4.0 (7.0)
Epoch 011/020 | Loss 0.0628 | Win/lose count 4.0/4.0 (0.0)
Epoch 012/020 | Loss 0.0024 | Win/lose count 8.5/3.0 (5.5)
Epoch 013/020 | Loss 0.0021 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/020 | Loss 0.0064 | Win/lose count 12.0/5.0 (7.0)
Epoch 015/020 | Loss 0.0057 | Win/lose count 12.0/4.0 (8.0)
Epoch 016/020 | Loss 0.0044 | Win/lose count 4.0/4

The other architectures tried were
1. 1 hidden layer with output size 128
2. 2 hidden layers with output size 256 and 128

The model with just 1 hidden layer and a lower learning rate seemed to work out well and is the one I have used for 20 epochs in the above run 

| Optimizer | Train Epochs | Learning rate | Epsilon | Architecture | Final Score |
| --- | --- | --- |
| SGD | 10 | .1 | 0.1 | 1 | 0.45
| SGD | 10 | .01 | 0.1 | 1 | -1.95
| SGD | 10 | .1 | 0.2 | 1 | -1.45
| Adam | 10 | .1 | 0.1 | 1 | -2.45
| Adam | 10 | .1 | 0.5 | 1 | -3.55
| **Adam** | **10** | **.01** | **0.1** | **1** | **2.25**
| Adam | 10 | .01 | 0.2 | 1 | 1.5
| Adam | 10 | .01 | 0.1 | 2 | 0.2
| SGD | 10 | .01 | 0.1 | 2 | -1.25
| SGD | 10 | .01 | 0.2 | 2 | -0.8
| SGD | 10 | .1 | 0.1 | 2 | -0.5

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [12]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(32,(2,2),activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4))
        #model.compile(loss='mse', optimizer=Adam(lr=lr))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [19]:

epochs_train=20
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0135 | Win/lose count 7.5/8.0 (-0.5)
Epoch 001/020 | Loss 0.0057 | Win/lose count 6.0/3.0 (3.0)
Epoch 002/020 | Loss 0.0145 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/020 | Loss 0.0046 | Win/lose count 5.0/3.0 (2.0)
Epoch 004/020 | Loss 0.0103 | Win/lose count 5.5/4.0 (1.5)
Epoch 005/020 | Loss 0.0137 | Win/lose count 7.0/3.0 (4.0)
Epoch 006/020 | Loss 0.0117 | Win/lose count 4.0/2.0 (2.0)
Epoch 007/020 | Loss 0.0148 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/020 | Loss 0.0128 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/020 | Loss 0.0123 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/020 | Loss 0.0095 | Win/lose count 7.5/3.0 (4.5)
Epoch 011/020 | Loss 0.0096 | Win/lose count 2.5/2.0 (0.5)
Epoch 012/020 | Loss 0.0109 | Win/lose count 7.0/1.0 (6.0)
Epoch 013/020 | Loss 0.0122 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/020 | Loss 0.0135 | Win/lose count 8.5/3.0 (5.5)
Epoch 015/020 | Loss 0.0408 | Win/lose count 7.0/3.0 (4.0)
Epoch 016/020 | Loss 0.0067 | Win/lose count 7.5/2.0 

Architectures tried
1. 2 Convolutional layers 50,(2,2) and 30,(2,2) with relu activation
2. Same architecture as 1 with dropout
3. 2 Convolutional layers 64,(2,2) and 32,(2,2) with relu activation
4. Same as 3 wth dropout
5. 2 Convolutional layers 64,(2,2) with relu activation
6. Same as 5 with dropout
7. 2 Convolutional layers 32,(2,2) with relu activation
8. Same as 7 with dropout

| Optimizer | Train Epochs | Learning rate | Epsilon | Architecture | Final Score |
| --- | --- | --- |
| SGD | 20 | .1 | 0.1 | 1 | 3.3
| SGD | 20 | .1 | 0.1 | 2 | 3.75
| SGD | 20 | .1 | 0.1 | 4 | 2.65
| SGD | 20 | .1 | 0.1 | 3 | 5.65
| SGD | 20 | .1 | 0.2 | 3 | 3.65
| SGD | 20 | .1 | 0.5 | 3 | 1.75
| SGD | 20 | .1 | 0.1 | 5 | 2.65
| SGD | 20 | .1 | 0.1 | 6 | 0.95
| SGD | 20 | .1 | 0.1 | 8 | 7.3


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [43]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.0/0. Average score (12.0)
Win/lose count 9.0/0. Average score (10.5)
Win/lose count 7.5/0. Average score (9.5)
Win/lose count 8.0/0. Average score (9.125)
Win/lose count 11.0/0. Average score (9.5)
Win/lose count 8.5/0. Average score (9.333333333333334)
Win/lose count 9.0/0. Average score (9.285714285714286)
Win/lose count 14.5/0. Average score (9.9375)
Win/lose count 2.0/1.0. Average score (8.944444444444445)
Win/lose count 4.0/0. Average score (8.45)
Final score: 8.45
Test of the FC
Win/lose count 6.0/0. Average score (6.0)
Win/lose count 3.0/0. Average score (4.5)
Win/lose count 0.5/0. Average score (3.1666666666666665)
Win/lose count 1.0/0. Average score (2.625)
Win/lose count 3.0/0. Average score (2.7)
Win/lose count 2.0/0. Average score (2.5833333333333335)
Win/lose count 2.0/0. Average score (2.5)
Win/lose count 2.0/0. Average score (2.4375)
Win/lose count 2.5/0. Average score (2.4444444444444446)
Win/lose count 0.5/0. Average score (2.25)
Final

In [44]:
HTML(display_videos('cnn_test0.mp4'))

In [45]:
HTML(display_videos('fc_test0.mp4'))

**Architecture**:The CNN performs much better than the fully connected model. However, it still tends to not explore the map as necessary. Increasing the value of epsilon to increase exploration does not improve the results.

**Temperature**:  The performance of both is drastically better when the temperature is increased. However, this is not an accurate representation of the score, as increasing the temperature means increasing the availability of cheese and descreasing the poisonous cells.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [29]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        #agent.set_epsilon(agent.epsilon*0.9)
        while not game_over:
            # The agent performs an action
            
            agent.set_epsilon(np.float(agent.epsilon*0.99))
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [30]:
# Training
epochs_train = 20
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.5, epsilon = 1, memory_size=10000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0106 | Win/lose count 6.50/28.60 (-22.10)
Epoch 001/020 | Loss 0.0040 | Win/lose count 2.00/18.10 (-16.10)
Epoch 002/020 | Loss 0.0113 | Win/lose count 4.50/20.40 (-15.90)
Epoch 003/020 | Loss 0.0084 | Win/lose count 14.50/15.30 (-0.80)
Epoch 004/020 | Loss 0.0145 | Win/lose count 15.00/14.50 (0.50)
Epoch 005/020 | Loss 0.0180 | Win/lose count 14.00/14.50 (-0.50)
Epoch 006/020 | Loss 0.0431 | Win/lose count 21.00/10.10 (10.90)
Epoch 007/020 | Loss 0.0121 | Win/lose count 16.00/14.60 (1.40)
Epoch 008/020 | Loss 0.0122 | Win/lose count 24.50/9.60 (14.90)
Epoch 009/020 | Loss 0.0103 | Win/lose count 25.00/13.50 (11.50)
Epoch 010/020 | Loss 0.0188 | Win/lose count 24.50/12.60 (11.90)
Epoch 011/020 | Loss 0.0080 | Win/lose count 21.50/13.70 (7.80)
Epoch 012/020 | Loss 0.0058 | Win/lose count 19.50/12.60 (6.90)
Epoch 013/020 | Loss 0.0156 | Win/lose count 20.50/10.60 (9.90)
Epoch 014/020 | Loss 0.0103 | Win/lose count 17.50/11.10 (6.40)
Epoch 015/020 | Loss 0.0092 | Wi

In [34]:
# Evaluation
epochs_test=10
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 22.0/16.99999999999998. Average score (5.000000000000021)
Win/lose count 24.0/13.499999999999973. Average score (7.750000000000024)
Win/lose count 21.0/14.699999999999964. Average score (7.266666666666695)
Win/lose count 6.5/17.999999999999986. Average score (2.575000000000025)
Win/lose count 20.0/12.89999999999997. Average score (3.480000000000026)
Win/lose count 15.5/15.999999999999961. Average score (2.8166666666666953)
Win/lose count 23.0/12.199999999999973. Average score (3.957142857142886)
Win/lose count 15.0/14.599999999999964. Average score (3.5125000000000295)
Win/lose count 14.0/15.79999999999996. Average score (2.9222222222222527)
Win/lose count 24.0/11.19999999999998. Average score (3.9100000000000295)
Final score: 3.9100000000000295


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***